 1- Build an RNN model 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence

In [256]:
## Load the data and Preprocess

with open("Ecoli_GCF_003018035.1_ASM301803v1_genomic.fna", "r") as file:
    data = file.read().splitlines()
    genome_sequence = "".join([line for line in data if not line.startswith('>')])

train_size = int(0.8 * len(genome_sequence))
train_data = genome_sequence[:train_size]
test_data = genome_sequence[train_size:]

char_to_idx = {char: i for i, char in enumerate(sorted(set(train_data)))}
idx_to_char = {i: char for char, i in char_to_idx.items()}

def sequence_to_tensor(sequence, char_to_idx):
    return torch.tensor([char_to_idx[char] for char in sequence], dtype=torch.long)

train_data_tensor = sequence_to_tensor(train_data, char_to_idx)
test_data_tensor = sequence_to_tensor(test_data, char_to_idx)

In [8]:
## Define RNN model architecture
class EcoliRNN(nn.Module):
    def __init__(self, inut_size, hidden_size, output_size):
        super(EcoliRNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

# Training parameters
input_size = len(char_to_idx)
hidden_size = 128
output_size = len(char_to_idx)
batch_size = 100
seq_length = 50
learning_rate = 0.0002
num_epochs = 2500

# Instantiate the model
model = EcoliRNN(input_size, hidden_size, output_size)
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [9]:
# Create input and target sequences for training
def random_training_example(train_data_tensor, seq_length):
    start_idx = torch.randint(0, len(train_data_tensor) - seq_length - 1, (1,)).item()
    input_sequence = train_data_tensor[start_idx:start_idx + seq_length]
    target_sequence = train_data_tensor[start_idx + 1:start_idx + seq_length + 1]
    return input_sequence, target_sequence

In [10]:
# One-hot encoding
def one_hot_encode(sequence, vocab_size):
    encoded = torch.zeros(len(sequence), vocab_size)
    for i, idx in enumerate(sequence):
        encoded[i, idx] = 1.0
    return encoded

# Training loop
for epoch in range(num_epochs):
    total_loss = 0
    for _ in range(batch_size):
        input_sequence, target_sequence = random_training_example(train_data_tensor, seq_length)
        input_sequence = one_hot_encode(input_sequence, input_size).unsqueeze(0)  # Add one-hot encoding and batch dimension
        hidden = model.init_hidden()
        model.zero_grad()

        loss = 0
        for i in range(input_sequence.size(1)):
            output, hidden = model(input_sequence[:, i], hidden)
            loss += criterion(output, target_sequence[i].unsqueeze(0))

        loss.backward()
        optimizer.step()
        total_loss += loss.item() / seq_length

    if epoch % 250 == 0:
        print(f"Epoch {epoch}: Loss {total_loss / batch_size}")

Epoch 0: Loss 1.384377044677734
Epoch 250: Loss 1.3544420898437501
Epoch 500: Loss 1.3570636283874506
Epoch 750: Loss 1.3541837188720702
Epoch 1000: Loss 1.359039141845703
Epoch 1250: Loss 1.352267694091797
Epoch 1500: Loss 1.3538534706115724
Epoch 1750: Loss 1.3561255218505857
Epoch 2000: Loss 1.357295195007324
Epoch 2250: Loss 1.3502745590209964


In [12]:
# Test the model on the test dataset
def evaluate(input_tensor, seq_length):
    with torch.no_grad():
        hidden = model.init_hidden()
        output_sequence = []

        for i in range(input_tensor.size(1)):
            output, hidden = model(input_tensor[:, i], hidden)
            top_idx = output.argmax(1).item()
            output_sequence.append(idx_to_char[top_idx])

        return "".join(output_sequence)

test_input, _ = random_training_example(test_data_tensor, seq_length)
test_input = one_hot_encode(test_input, input_size).unsqueeze(0)  # Adding a one-hot encoding and batch dimension
predicted_sequence = evaluate(test_input, seq_length)
print("Predicted sequence:", predicted_sequence)

Predicted sequence: CAGTAACTATAGGCAGAAGAGGTACGACGATGATCAGCAGCAGGGTCCCG


In [21]:
# Modified evaluate function to return accuracy
def evaluate(input_tensor, target_tensor):
    with torch.no_grad():
        hidden = model.init_hidden()
        correct = 0

        for i in range(input_tensor.size(1)):
            output, hidden = model(input_tensor[:, i], hidden)
            top_idx = output.argmax(1).item()
            if top_idx == target_tensor[i].item():
                correct += 1

        accuracy = correct / input_tensor.size(1)
        return accuracy

# Calculate average test accuracy
num_test_examples = 500000
total_accuracy = 0.0

for _ in range(num_test_examples):
    test_input, test_target = random_training_example(test_data_tensor, seq_length)
    test_input = one_hot_encode(test_input, input_size).unsqueeze(0)
    accuracy = evaluate(test_input, test_target)
    total_accuracy += accuracy

avg_test_accuracy = total_accuracy / num_test_examples
print(f"Average test accuracy: {avg_test_accuracy * 100:.2f}%")

Average test accuracy: 32.62%


2-Build a word-level model using RNN and Break a DNA sequence into k-mers

In [257]:
# Preprocessing
def generate_kmers(sequence, k):
    return [sequence[i:i+k] for i in range(len(sequence) - k + 1)]

k = 4
train_kmers = generate_kmers(train_data, k)
test_kmers = generate_kmers(test_data, k)

unique_kmers = sorted(set(train_kmers))
kmer_to_idx = {kmer: i for i, kmer in enumerate(unique_kmers)}
idx_to_kmer = {i: kmer for kmer, i in kmer_to_idx.items()}

vocab_size = len(unique_kmers)


In [210]:
# new Model
class KmerRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(KmerRNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

In [211]:
# Training and evaluate
import random

def random_training_example(data, kmer_to_idx):
    kmer_idx = random.randint(0, len(data) - 2)
    input_kmer = data[kmer_idx]
    target_kmer = data[kmer_idx + 1]
    return torch.tensor([kmer_to_idx[input_kmer]], dtype=torch.long), torch.tensor([kmer_to_idx[target_kmer]], dtype=torch.long)

def one_hot_encode(sequence, vocab_size):
    encoded = torch.zeros(1, vocab_size)
    for idx in sequence:
        encoded[0, idx] = 1.0
    return encoded

def train(input_tensor, target_tensor, model, criterion, optimizer):
    hidden = model.init_hidden()
    model.zero_grad()
    loss = 0

    output, hidden = model(input_tensor, hidden)
    loss = criterion(output, target_tensor)

    loss.backward()
    optimizer.step()

    return loss.item()

In [212]:
# New Training parameter
input_size = vocab_size
hidden_size = 128
output_size = vocab_size

model = KmerRNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10000
batch_size = 100

In [57]:
# Training loop
for epoch in range(num_epochs):
    total_loss = 0
    for _ in range(batch_size):
        input_kmer, target_kmer = random_training_example(train_kmers, kmer_to_idx)
        input_tensor = one_hot_encode(input_kmer, input_size)
        loss = train(input_tensor, target_kmer, model, criterion, optimizer)
        total_loss += loss

    if epoch % 250 == 0:
        print(f"Epoch {epoch}: Loss {total_loss / batch_size}")

def test_model(input_tensor, target_tensor, model):
    with torch.no_grad():
        hidden = model.init_hidden()
        output, _ = model(input_tensor, hidden)
        top_idx = output.argmax(1).item()

        accuracy = 1 if top_idx == target_tensor.item() else 0
        predicted_kmer = idx_to_kmer[top_idx]
        actual_kmer = idx_to_kmer[target_tensor.item()]

        return accuracy, predicted_kmer, actual_kmer

Epoch 0: Loss 5.547083258628845
Epoch 250: Loss 3.6827583360671996
Epoch 500: Loss 2.248655087351799
Epoch 750: Loss 1.597973136305809
Epoch 1000: Loss 1.5414057582616807
Epoch 1250: Loss 1.426462857723236
Epoch 1500: Loss 1.4123170167207717
Epoch 1750: Loss 1.3980057382583617
Epoch 2000: Loss 1.3772445595264435
Epoch 2250: Loss 1.384626290202141
Epoch 2500: Loss 1.4389013969898223
Epoch 2750: Loss 1.3612875956296921
Epoch 3000: Loss 1.3069980251789093
Epoch 3250: Loss 1.418328297138214
Epoch 3500: Loss 1.2573722660541535
Epoch 3750: Loss 1.4146320909261703
Epoch 4000: Loss 1.4376247972249985
Epoch 4250: Loss 1.4022792196273803
Epoch 4500: Loss 1.4348552185297012
Epoch 4750: Loss 1.3971243649721146
Epoch 5000: Loss 1.3482069471478462
Epoch 5250: Loss 1.337926074564457
Epoch 5500: Loss 1.3567278790473938
Epoch 5750: Loss 1.3819846951961516
Epoch 6000: Loss 1.332816428542137
Epoch 6250: Loss 1.3065741264820099
Epoch 6500: Loss 1.2985156771540642
Epoch 6750: Loss 1.3082764089107513
Epoch 

In [61]:
# define a Hamming distance between two strings s1 and s2 of equal length
def hamming_distance(s1, s2):
    return sum(el1 != el2 for el1, el2 in zip(s1, s2))

def nuanced_accuracy(predicted_kmer, actual_kmer):
    return 1 - (hamming_distance(predicted_kmer, actual_kmer) / len(predicted_kmer))

input_seq = "ATCGAGGATC"
kmers = generate_kmers(input_seq, k)

print("Input sequence:", input_seq)
print("K-mers in the input sequence:", kmers)

total_accuracy = 0.0
total_nuanced_accuracy = 0.0

for i in range(len(kmers) - 1):
    input_kmer = torch.tensor([kmer_to_idx[kmers[i]]], dtype=torch.long)
    input_kmer_tensor = one_hot_encode(input_kmer, input_size)
    target_kmer = torch.tensor([kmer_to_idx[kmers[i + 1]]], dtype=torch.long)

    accuracy, predicted_kmer, actual_kmer = test_model(input_kmer_tensor, target_kmer, model)
    total_accuracy += accuracy
    
    nuanced_acc = nuanced_accuracy(predicted_kmer, actual_kmer)
    total_nuanced_accuracy += nuanced_acc

    print(f"Example {i + 1}:")
    print(f"Input K-mer: {kmers[i]}")
    print(f"Predicted K-mer: {predicted_kmer}")
    print(f"Actual K-mer: {actual_kmer}")
    print(f"Strict Accuracy: {accuracy * 100}%")
    print(f"Nuanced Accuracy: {nuanced_acc * 100}%\n")

avg_test_accuracy = total_accuracy / (len(kmers) - 1)
avg_nuanced_test_accuracy = total_nuanced_accuracy / (len(kmers) - 1)
print(f"Average strict test accuracy: {avg_test_accuracy * 100:.2f}%")
print(f"Average nuanced test accuracy: {avg_nuanced_test_accuracy * 100:.2f}%")

Input sequence: ATCGAGGATC
K-mers in the input sequence: ['ATCG', 'TCGA', 'CGAG', 'GAGG', 'AGGA', 'GGAT', 'GATC']
Example 1:
Input K-mer: ATCG
Predicted K-mer: TCGC
Actual K-mer: TCGA
Strict Accuracy: 0%
Nuanced Accuracy: 75.0%

Example 2:
Input K-mer: TCGA
Predicted K-mer: CGAT
Actual K-mer: CGAG
Strict Accuracy: 0%
Nuanced Accuracy: 75.0%

Example 3:
Input K-mer: CGAG
Predicted K-mer: GAGA
Actual K-mer: GAGG
Strict Accuracy: 0%
Nuanced Accuracy: 75.0%

Example 4:
Input K-mer: GAGG
Predicted K-mer: AGGA
Actual K-mer: AGGA
Strict Accuracy: 100%
Nuanced Accuracy: 100.0%

Example 5:
Input K-mer: AGGA
Predicted K-mer: GGAA
Actual K-mer: GGAT
Strict Accuracy: 0%
Nuanced Accuracy: 75.0%

Example 6:
Input K-mer: GGAT
Predicted K-mer: GATG
Actual K-mer: GATC
Strict Accuracy: 0%
Nuanced Accuracy: 75.0%

Average strict test accuracy: 16.67%
Average nuanced test accuracy: 79.17%


3-RNN with Attention design to improve the model

In [258]:
import torch.nn.functional as F

class KmerRNNAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(KmerRNNAttention, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.attn = nn.Linear(hidden_size, input_size)  # Attention layer

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)

        # Attention mechanism
        attn_weights = F.softmax(self.attn(hidden), dim=1)
        attn_applied = torch.mul(attn_weights, input)

        output = self.i2o(attn_applied)
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

In [259]:
def random_training_example(data, kmer_to_idx):
    kmer_idx = random.randint(0, len(data) - 2)
    input_kmer = data[kmer_idx]
    target_kmer = data[kmer_idx + 1]
    return torch.tensor([kmer_to_idx[input_kmer]], dtype=torch.long), torch.tensor([kmer_to_idx[target_kmer]], dtype=torch.long)

def one_hot_encode(sequence, vocab_size):
    encoded = torch.zeros(1, vocab_size)
    for idx in sequence:
        encoded[0, idx] = 1.0
    return encoded

def train(input_tensor, target_tensor, model, criterion, optimizer):
    hidden = model.init_hidden()
    model.zero_grad()
    loss = 0

    output, hidden = model(input_tensor, hidden)
    loss = criterion(output, target_tensor)

    loss.backward()
    optimizer.step()

    return loss.item()

In [262]:
# New Training parameter
input_size = vocab_size
hidden_size = 256
output_size = vocab_size

model = KmerRNNAttention(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5000
batch_size = 100

In [263]:
# Training loop
for epoch in range(num_epochs):
    total_loss = 0
    for _ in range(batch_size):
        input_kmer, target_kmer = random_training_example(train_kmers, kmer_to_idx)
        input_tensor = one_hot_encode(input_kmer, input_size)
        loss = train(input_tensor, target_kmer, model, criterion, optimizer)
        total_loss += loss

    if epoch % 250 == 0:
        print(f"Epoch {epoch}: Loss {total_loss / batch_size}")

Epoch 0: Loss 5.5569839811325075
Epoch 250: Loss 3.4967409777641296
Epoch 500: Loss 2.3014929807186126
Epoch 750: Loss 1.6671909499168396
Epoch 1000: Loss 1.5686285656690597
Epoch 1250: Loss 1.336796681880951
Epoch 1500: Loss 1.3385279452800751
Epoch 1750: Loss 1.3203246515989304
Epoch 2000: Loss 1.3593204373121262
Epoch 2250: Loss 1.3141004472970963
Epoch 2500: Loss 1.3457167062163353
Epoch 2750: Loss 1.4145583122968675
Epoch 3000: Loss 1.3378634890913963
Epoch 3250: Loss 1.3982258999347688
Epoch 3500: Loss 1.3437099224328994
Epoch 3750: Loss 1.4484151017665863
Epoch 4000: Loss 1.429296840429306
Epoch 4250: Loss 1.4316371649503707
Epoch 4500: Loss 1.4179669213294983
Epoch 4750: Loss 1.3864009046554566


In [270]:
#testing set 

def test_model_attention(input_tensor, target_tensor, model):
    with torch.no_grad():
        hidden = model.init_hidden()
        output, _ = model(input_tensor, hidden)
        top_idx = output.argmax(1).item()

        accuracy = 1 if top_idx == target_tensor.item() else 0
        predicted_kmer = idx_to_kmer[top_idx]
        actual_kmer = idx_to_kmer[target_tensor.item()]

        return accuracy, predicted_kmer, actual_kmer

def hamming_distance(s1, s2):
    return sum(el1 != el2 for el1, el2 in zip(s1, s2))

def nuanced_accuracy(predicted_kmer, actual_kmer):
    return 1 - (hamming_distance(predicted_kmer, actual_kmer) / len(predicted_kmer))

input_seq = "ATCGAGGATC"
kmers = generate_kmers(input_seq, k)

print("Input sequence:", input_seq)
print("K-mers in the input sequence:", kmers)

total_accuracy = 0.0
total_nuanced_accuracy = 0.0

for i in range(len(kmers) - 1):
    input_kmer = torch.tensor([kmer_to_idx[kmers[i]]], dtype=torch.long)
    input_kmer_tensor = one_hot_encode(input_kmer, input_size)
    target_kmer = torch.tensor([kmer_to_idx[kmers[i + 1]]], dtype=torch.long)

    accuracy, predicted_kmer, actual_kmer = test_model_attention(input_kmer_tensor, target_kmer, model)
    total_accuracy += accuracy
    
    nuanced_acc = nuanced_accuracy(predicted_kmer, actual_kmer)
    total_nuanced_accuracy += nuanced_acc

    print(f"Example {i + 1}:")
    print(f"Input K-mer: {kmers[i]}")
    print(f"Predicted K-mer: {predicted_kmer}")
    print(f"Actual K-mer: {actual_kmer}")
    print(f"Strict Accuracy: {accuracy * 100}%")
    print(f"Nuanced Accuracy: {nuanced_acc * 100}%\n")

avg_test_accuracy = total_accuracy / (len(kmers) - 1)
avg_nuanced_test_accuracy = total_nuanced_accuracy / (len(kmers) - 1)
print(f"Average strict test accuracy: {avg_test_accuracy * 100:.2f}%")
print(f"Average nuanced test accuracy: {avg_nuanced_test_accuracy * 100:.2f}%")

Input sequence: ATCGAGGATC
K-mers in the input sequence: ['ATCG', 'TCGA', 'CGAG', 'GAGG', 'AGGA', 'GGAT', 'GATC']
Example 1:
Input K-mer: ATCG
Predicted K-mer: TCGC
Actual K-mer: TCGA
Strict Accuracy: 0%
Nuanced Accuracy: 75.0%

Example 2:
Input K-mer: TCGA
Predicted K-mer: CGAT
Actual K-mer: CGAG
Strict Accuracy: 0%
Nuanced Accuracy: 75.0%

Example 3:
Input K-mer: CGAG
Predicted K-mer: GAGC
Actual K-mer: GAGG
Strict Accuracy: 0%
Nuanced Accuracy: 75.0%

Example 4:
Input K-mer: GAGG
Predicted K-mer: AGGA
Actual K-mer: AGGA
Strict Accuracy: 100%
Nuanced Accuracy: 100.0%

Example 5:
Input K-mer: AGGA
Predicted K-mer: GGAA
Actual K-mer: GGAT
Strict Accuracy: 0%
Nuanced Accuracy: 75.0%

Example 6:
Input K-mer: GGAT
Predicted K-mer: GATG
Actual K-mer: GATC
Strict Accuracy: 0%
Nuanced Accuracy: 75.0%

Average strict test accuracy: 16.67%
Average nuanced test accuracy: 79.17%
